## Streaming_Architecture

In [4]:
from pyspark.sql import SparkSession

In [ ]:
import json
import random
import os
import time  

# 1. 저장 경로
directory = "data/streaming_sample"
if not os.path.exists(directory):
    os.makedirs(directory)

# 2. 재료 준비
cities = ["Seoul", "New York", "London", "Paris", "Tokyo", "Busan"]
events = ["view", "click", "purchase", "login", "logout", "add_to_cart"]
domains = ["google.com", "naver.com", "yahoo.com", "facebook.com", "medium.com"]

#  파일 이름에 시간을 붙임
# 예: sample_1706934212.json
current_time = int(time.time()) 
file_path = f"{directory}/sample_{current_time}.json"

print(f"📂 '{directory}' 폴더에 새로운 파일을 생성 중입니다...")

# 4. 파일 쓰기
with open(file_path, "w", encoding="utf-8") as f:
    for _ in range(100): 
        data = {
            "city": random.choice(cities),
            "domain": random.choice(domains),
            "event": random.choice(events)
        }
        f.write(json.dumps(data) + "\n")

print(f"✅ 생성 완료! 이번 파일: sample_{current_time}.json")
print("👉 스파크가 이 '새로운 파일'을 감지하고 가져갈 것입니다!")

In [5]:
spark=(
    SparkSession
    .builder
    .appName("StrucuredStreamingSum")
    .config("spark.streaming.stopGracefullyOnShutdown","true")
    .config("spark.sql.streaming.schemaInference","true")
    .config("spark.ui.port","4041")
    .config("maxFilesPerTrigger",1)
    .getOrCreate()
)

df=(
    spark
    .readStream
    .format("json")
    .option("path","data/streaming_sample")
    .load()
)
shorten_df=df.select("city","event")

query=(
    shorten_df
    .writeStream
    .format("json")
    .option("path","data/streaming_output")
    .option("checkpointLocation","checkpoint")
    .outputMode("append")
    .trigger(processingTime='5 seconds')
    .start()
)
query.awaitTermination()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/03 05:58:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/03 05:58:17 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
RuntimeError: reentrant call inside <_io.BufferedReader name=58>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.8/dist-

Py4JError: An error occurred while calling o49.awaitTermination

In [15]:
# 주피터 노트북 셀에서 실행해서 폴더 삭제 (리셋)
import shutil
import os
cwd = os.getcwd()


# 기록 지우기
if os.path.exists("checkpoint"):
    shutil.rmtree("checkpoint")
    print("✅ 체크포인트 삭제 완료 (기억 리셋)")

if os.path.exists("data/streaming_output"):
    shutil.rmtree("data/streaming_output")
    print("✅ 결과 폴더 삭제 완료 (새 출발)")